In [2]:
import os, shutil

import keras
import numpy as np

from PIL import Image

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt

from matplotlib.colors import LinearSegmentedColormap, ListedColormap
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import FormatStrFormatter

## Change these values

In [3]:
root = r"/Users/robertocastello/deneb/labelling_tool" # The root of your git folder
folder_verify = "images_valid" # The folder containing images to verify (images_valid or images_test)
destination_folders = ['images_relabel', 'images_ignore'] # The folders to move images to

# --------------

In [4]:
for folder in destination_folders:
    for subfolder in ["labels", "originals", "noPV"]:
        fullfolder = os.path.join(root, folder, subfolder)
        if not os.path.isdir(fullfolder):
            os.makedirs(fullfolder)

In [5]:
def load_eval_images(x_eval_dir, y_eval_dir, nopv_dir = None, valid = False):
#     x_eval_dir = r"C:\Users\SimonRoquette\Documents\solarPV\new_test_images/PV/originals"
#     y_eval_dir = r"C:\Users\SimonRoquette\Documents\solarPV\new_test_images/PV/labels/"
#     nopv_dir = r"C:\Users\SimonRoquette\Documents\solarPV\images_test\noPV"

    x_image_paths = [os.path.join(x_eval_dir, f) for f in os.listdir(x_eval_dir)]
    x_eval = np.array([np.array(Image.open(f), dtype='uint8') for f in x_image_paths])
    x_eval_nopad = x_eval
    x_eval = np.pad(x_eval, ((0, 0), (3, 3), (3,3), (0, 0)), mode="constant", constant_values=0)

    y_image_paths = [os.path.join(os.path.dirname(os.path.dirname(f)), "labels", 
                                   os.path.basename(f).replace(".png", "_label.png")) for f in x_image_paths]
    y_eval = [np.array(Image.open(f), dtype='uint8') for f in y_image_paths]
    y_eval = np.array([a[:, :, 0] if len(a.shape) == 3 else a for a in y_eval])
    y_eval_nopad = np.array(y_eval)
    if valid:
        y_eval = np.pad(y_eval, ((0, 0), (3, 3), (3,3)), mode="constant", constant_values=0)
    y_eval = np.stack((np.where(y_eval == 0, 1, 0),
                        y_eval), axis=-1)
    y_eval = y_eval.astype(np.bool)
   
    if nopv_dir is not None:
        image_paths = [os.path.join(nopv_dir, f) for f in os.listdir(nopv_dir)]
        x_nopv = np.array([np.array(Image.open(f), dtype='uint8') for f in image_paths])
        x_nopv = np.pad(x_nopv, ((0, 0), (3, 3), (3,3), (0, 0)), mode="constant", constant_values=0)

        side = 256 if valid else 250
        y_nopv = np.stack((np.ones((len(x_nopv), side, side), dtype=np.bool),
                           np.zeros((len(x_nopv), side, side), dtype=np.bool)),
                          axis=-1)
        
        print("Loaded %i Pv images, and %i non PV" % (len(x_eval), len(x_nopv)))
        
        # Including noPV
        X_test = np.concatenate((x_eval, x_nopv))
        Y_test = np.concatenate((y_eval, y_nopv)).astype("float32")
        x_image_paths = x_image_paths + image_paths
    else :
        print("Loaded %i PV images" % (len(x_eval)))
        X_test = x_eval
        Y_test = y_eval.astype("float32")
    
    return X_test, Y_test, x_image_paths

In [6]:
x_test_dir = os.path.join(root, folder_verify, "originals")
y_test_dir = os.path.join(root, folder_verify, "labels")
test_nopv_dir = os.path.join(root, folder_verify, "noPV")

X_test, Y_test, test_image_paths = load_eval_images(x_test_dir, y_test_dir, test_nopv_dir)

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/Users/robertocastello/deneb/labelling_tool\\images_valid\\originals'

In [ ]:
ops = []

def on_button_click_i(directory_dest, image, selector, noPV= False):
    def on_button_click(b):
        
        if noPV:
            from_original = os.path.join(root, folder_verify, "noPV", image)
            to_original = os.path.join(root, directory_dest, "noPV", image)
            shutil.move(from_original, to_original)
            ops.append((from_original, to_original))
        else :  
            labelimage = image.replace(".png", "_label.png")

            from_original = os.path.join(root, folder_verify, "originals", image)
            to_original = os.path.join(root, directory_dest, "originals", image)

            from_label = os.path.join(root, folder_verify, "labels", labelimage)
            to_label = os.path.join(root, directory_dest, "labels" , labelimage)

            shutil.move(from_original, to_original)
            ops.append((from_original, to_original))

            shutil.move(from_label, to_label)
            ops.append((from_label, to_label))
        
        print("Moved :", image)
#         print("From :", os.path.join(root, r"SI_25_classes\all_pv", image))
#         print("To : ", os.path.join(root, directory_dest, "originals", image))
    return on_button_click

def on_click_cancel(b, ops=ops):
    op1 = ops[-2]
    op2 = ops[-1]
    
    shutil.move(op1[1], op1[0])
    print("Moved :", op1[1])
    print("Back to :", op1[0])
    
    shutil.move(op2[1], op2[0])
    print("Moved :", op2[1])
    print("Back to :", op2[0])
          
    del ops[-1]
    del ops[-1]

def on_button_update(selector, value = 1):
    def on_button_click(b):
        selector.value = selector.value + value
    return on_button_click

In [ ]:
cmaplabel = LinearSegmentedColormap.from_list('mycmap', [(0,1, 0,c) for c in np.linspace(0,1,100)], N=5)
norm = mpl.colors.Normalize(vmin=-2, vmax=1)
colors = ['b','r','none', "g"]
cmap = ListedColormap(colors)
m = cm.ScalarMappable(norm=norm, cmap=cmap)

beg_selector = widgets.IntText(
                        value=0,
                        description='Image :',
                        disabled=False
                    )

directory_destination = widgets.Dropdown(
    options= destination_folders,
    value=destination_folders[0],
    description='Dest:',
    disabled=False
)

button_move = widgets.Button(
    description='Move to Dest',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='' # (FontAwesome names without the `fa-` prefix)
)

button_keep = widgets.Button(
    description='Keep in current folder',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='' # (FontAwesome names without the `fa-` prefix)
)

button_undo = widgets.Button(
    description='Undo Previous',
    disabled=False,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='' # (FontAwesome names without the `fa-` prefix)
)

button_undo.on_click(on_click_cancel)
button_undo.on_click(on_button_update(beg_selector, -1))

button_keep.on_click(on_button_update(beg_selector, 1))

display(button_move, button_keep, button_undo)

@interact
def interactive_image_threshold(i = beg_selector, dir_dest = directory_destination):
    noPV = "noPV" in test_image_paths[i]
    
    imagename = os.path.basename(test_image_paths[i])
    button_move._click_handlers.callbacks = [on_button_update(beg_selector, 1)]
    button_move.on_click(on_button_click_i(dir_dest, imagename, beg_selector, noPV))

    fig, ax = plt.subplots(1, 2, figsize=(16, 16))
    ax[0].imshow(X_test[i][3:-3, 3:-3, :])

    # Color Bar
    divider = make_axes_locatable(ax[1])
    cax = divider.append_axes("right", size="7%", pad=0.05)
    cbar = plt.colorbar(m, cax=cax)
    cbar.ax.get_yaxis().set_ticks([])
    for j, lab in enumerate(['FN','FP','TN','TP']):
        cbar.ax.text(-0.5, (4 * (j-2)) / 6, lab, ha='center', va='center', fontweight="bold",
                     color = "white" if lab != "TN" else "black")
    cbar.ax.get_yaxis().labelpad = 15
    cbar.ax.set_ylabel('Truth', rotation=270)

    print(imagename)

    ax[1].imshow(X_test[i][3:-3, 3:-3, :])
    ax[1].imshow(Y_test[i, :, :, 1], cmap = cmaplabel)
    ax[1].set_title(("Label (i : %i)" % i))

In [ ]:
test_image_paths[0]

In [ ]:
test_image_paths[0]